In [11]:
%load_ext autoreload
%autoreload 2
from lstm import *
from s3 import S3Client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
s3 = S3Client()
s3.download_file('models/lstm_best.pt', 'lstm_best.pt')
s3.download_file('models/lstm_best_startup_params.json', 'lstm_best_startup_params.json')

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [8]:
s3 = S3Client()

In [4]:
import datetime


today = datetime.datetime.today()
today_unix = int(today.timestamp())
one_week_ago = today - datetime.timedelta(days=7)

one_week_ago

def unix_ago(seconds = 0, hours = 0, days = 0):
    now = datetime.datetime.now()
    one_week_ago = now - datetime.timedelta(seconds=seconds, hours=hours, days=days)
    return int(one_week_ago.timestamp())


In [33]:
products = ['BTC-USD']
start_datetime = 
end_datetime = 
granularity = 'FIVE_MINUTE'

granularity_to_seconds = {
      'ONE_MINUTE': 60,
      'FIVE_MINUTE': 60 * 5,
      'FIFTEEN_MINUTE': 60 * 15,
      'THIRTY_MINUTE': 60 * 30,
      'ONE_HOUR': 60 * 60,
      'TWO_HOUR': 60 * 60 * 2,
      'SIX_HOUR': 60 * 60 * 6,
      'ONE_DAY': 60 * 60 * 24
  }

In [36]:
results = []

MAX_POINTS = 300


Getting candles from 2023-12-01 00:00:00 to 2023-12-02 01:00:00
Getting candles from 2023-12-02 01:00:00 to 2023-12-03 02:00:00
Getting candles from 2023-12-03 02:00:00 to 2023-12-04 03:00:00
Getting candles from 2023-12-04 03:00:00 to 2023-12-05 04:00:00
Getting candles from 2023-12-05 04:00:00 to 2023-12-06 05:00:00
Getting candles from 2023-12-06 05:00:00 to 2023-12-07 06:00:00
Getting candles from 2023-12-07 06:00:00 to 2023-12-08 07:00:00
Getting candles from 2023-12-08 07:00:00 to 2023-12-09 08:00:00
Getting candles from 2023-12-09 08:00:00 to 2023-12-10 09:00:00
Getting candles from 2023-12-10 09:00:00 to 2023-12-11 10:00:00
Getting candles from 2023-12-11 10:00:00 to 2023-12-12 11:00:00
Getting candles from 2023-12-12 11:00:00 to 2023-12-13 12:00:00
Getting candles from 2023-12-13 12:00:00 to 2023-12-14 13:00:00
Getting candles from 2023-12-14 13:00:00 to 2023-12-15 14:00:00
Getting candles from 2023-12-15 14:00:00 to 2023-12-16 15:00:00
Getting candles from 2023-12-16 15:00:00

In [35]:
etime

datetime.datetime(2023, 12, 2, 1, 0)

In [65]:
x

{'error': 'INVALID_ARGUMENT',
 'error_details': 'start and end argument is invalid - number of candles requested should be less than 300 ',
 'message': 'start and end argument is invalid - number of candles requested should be less than 300 '}

In [ ]:
import time
import time

# Get today's time in Unix
today_unix = int(time.time())

# Get one week ago's time in Unix
one_week_ago_unix = today_unix - (7 * 24 * 60 * 60)

today_unix, one_week_ago_unix


products.getProductCandles(
    'BTC-USD', 
    start = 
)